In [ ]:
from nltk.corpus import semcor
import re

In [ ]:
import nltk
nltk.download('semcor')
!unzip -o /root/nltk_data/corpora/semcor.zip -d /root/nltk_data/corpora

[nltk_data] Downloading package semcor to /root/nltk_data...
[nltk_data]   Package semcor is already up-to-date!
Archive:  /root/nltk_data/corpora/semcor.zip
  inflating: /root/nltk_data/corpora/semcor/brown1/tagfiles/br-a01.xml  
  inflating: /root/nltk_data/corpora/semcor/brown1/tagfiles/br-a02.xml  
  inflating: /root/nltk_data/corpora/semcor/brown1/tagfiles/br-a11.xml  
  inflating: /root/nltk_data/corpora/semcor/brown1/tagfiles/br-a12.xml  
  inflating: /root/nltk_data/corpora/semcor/brown1/tagfiles/br-a13.xml  
  inflating: /root/nltk_data/corpora/semcor/brown1/tagfiles/br-a14.xml  
  inflating: /root/nltk_data/corpora/semcor/brown1/tagfiles/br-a15.xml  
  inflating: /root/nltk_data/corpora/semcor/brown1/tagfiles/br-b13.xml  
  inflating: /root/nltk_data/corpora/semcor/brown1/tagfiles/br-b20.xml  
  inflating: /root/nltk_data/corpora/semcor/brown1/tagfiles/br-c01.xml  
  inflating: /root/nltk_data/corpora/semcor/brown1/tagfiles/br-c02.xml  
  inflating: /root/nltk_data/corpora/se

In [ ]:
import numpy as np

def get_context_embedding(sentence): #sentences = semcor.sents()
  sum = np.zeros(300,)
  no_of_words = 0

  for word in sentence:
    try:
      sum = sum + model_w2v[word]
      no_of_words = no_of_words + 1
    except:
      pass

  if no_of_words != 0:
    con_emb = sum/no_of_words
  else:
    con_emb = sum
  return con_emb

context_embeddings = []
sentences = semcor.sents()
for sentence in sentences:
  context_embeddings.append(get_context_embedding(sentence))

In [ ]:
tagged_sentences = [[str(c) for c in s] for s in semcor.tagged_sents(tag="sem")]
tagged_sentences[0]

["['The']",
 "(Lemma('group.n.01.group') (NE Fulton County Grand Jury))",
 "(Lemma('state.v.01.say') said)",
 "(Lemma('friday.n.01.Friday') Friday)",
 "['an']",
 "(Lemma('probe.n.01.investigation') investigation)",
 "['of']",
 "(Lemma('atlanta.n.01.Atlanta') Atlanta)",
 '["\'s"]',
 "(Lemma('late.s.03.recent') recent)",
 "(Lemma('primary.n.01.primary_election') primary election)",
 "(Lemma('produce.v.04.produce') produced)",
 "['``']",
 "['no']",
 "(Lemma('evidence.n.01.evidence') evidence)",
 '["\'\'"]',
 "['that']",
 "['any']",
 "(Lemma('abnormality.n.04.irregularity') irregularities)",
 "(Lemma('happen.v.01.take_place') took place)",
 "['.']"]

In [ ]:
filtered_sentences=[]
for sentence in tagged_sentences:
  filtered_sentence=[]
  for word in sentence:
    if not word.startswith('['):
      pair = tuple(re.sub(r'[\(\)]', '', word).split(' ', 1)[::-1])
      filtered_sentence.append(pair)
  filtered_sentences.append(filtered_sentence)

filtered_sentences[0]

[('NE Fulton County Grand Jury', "Lemma'group.n.01.group'"),
 ('said', "Lemma'state.v.01.say'"),
 ('Friday', "Lemma'friday.n.01.Friday'"),
 ('investigation', "Lemma'probe.n.01.investigation'"),
 ('Atlanta', "Lemma'atlanta.n.01.Atlanta'"),
 ('recent', "Lemma'late.s.03.recent'"),
 ('primary election', "Lemma'primary.n.01.primary_election'"),
 ('produced', "Lemma'produce.v.04.produce'"),
 ('evidence', "Lemma'evidence.n.01.evidence'"),
 ('irregularities', "Lemma'abnormality.n.04.irregularity'"),
 ('took place', "Lemma'happen.v.01.take_place'")]

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

#Import necessary modules and load word2vec
from gensim.models import KeyedVectors
model_w2v = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

import nltk
nltk.download('wordnet')

--2021-11-04 08:35:47--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.196.152
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.196.152|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def get_senses(phrase, POS=None): #pos=n, v, r, a, s
  words = phrase.split(" ")

  if len(words) == 1:
    synset = wn.synsets(phrase) 
    senses = [s.name() for s in synset]

  else:
    if words[0] == 'NE':
      senses = ['group.n.01', 'location.n.01', 'person.n.01']
    else:
      senses = []
      for word in words:
        synset = wn.synsets(word)
        temp = [s.name() for s in synset]
        senses.extend(temp)
      comb_word = "_".join(words)
      comb_sense = wn.synsets(comb_word)
      temp2 = [s.name() for s in comb_sense]
      senses.extend(temp2)

  senses1 = []
  if POS!=None:
    for se in senses:
      se1 = se.split(".")
      if se1[1]==POS:
        senses1.append(se)
    return senses1
  else:
    return senses

def get_sense_embedding(senses):
  output = {}
  # senses = wn.synsets(word)
  for sense in senses:
    gloss = wn.synset(sense).definition()
    gloss_words = gloss.split(" ")
    word_vector_sum = 0
    gloss_words_count = 0
    for word in gloss_words:
      try:
        word_vector_sum = word_vector_sum + model_w2v[word]
        gloss_words_count = gloss_words_count + 1
      except:
        pass
    try:
      sense_embedding = word_vector_sum/gloss_words_count
    except ZeroDivisionError:
      sense_embedding = np.zeros(300,)

    output[sense] = sense_embedding

  return output

In [ ]:
get_senses("international conference")

['international.n.01',
 'international.a.01',
 'external.s.03',
 'conference.n.01',
 'league.n.01',
 'conference.n.03']

In [ ]:
from nltk.corpus import wordnet as wn
from numpy import dot
from numpy.linalg import norm
import numpy as np

test_data = [[word for (word, _) in sentence] for sentence in filtered_sentences]
# test_data = [["went", "bank", "withdraw", "some", "money"]]
# true_tag_sequences = [[tag for (_, tag) in sentence] for sentence in filtered_sentences]

predicted_tags = []
index = 0
for sentence in test_data:
  sentence_tags = []
  context_embedding = context_embeddings[index]
  index = index + 1
  for phrase in sentence:
    senses = get_senses(phrase)
    sense_emb_list = get_sense_embedding(senses)

    best_sense = ''
    max_cos_sim = 0
    for sense, sense_embedding in sense_emb_list.items():  
      cos_sim = dot(sense_embedding, context_embedding)/(norm(sense_embedding)*norm(context_embedding))
      if cos_sim > max_cos_sim:
        best_sense = sense
        max_cos_sim = cos_sim
    
    if phrase.startswith('NE'):
      print(phrase, best_sense)

    sentence_tags.append(best_sense)

  predicted_tags.append(sentence_tags)

predicted_tags

[['belong.v.03', 'bank.n.07', 'swallow.v.05', 'some.s.02', 'money.n.03']]

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

# // Dummy text
# txt = "Sukanya, Rajib and Naba are my good friends. " \
# 	"Sukanya is getting married next year. " \
# 	"Marriage is a big step in one’s life." \
# 	"It is both exciting and frightening. " \
# 	"But friendship is a sacred bond between people." \
# 	"It is a special kind of love between us. " \
# 	"Many of you must have tried searching for a friend "\
# 	"but never found the right one."

# txt = "The school is seeing return of students. "
# txt = 'Elderly with young face increased covid19 risk'
txt = 'The old ash tree near the river stood tall and slender'
# sent_tokenize is one of instances of
# PunktSentenceTokenizer from the nltk.tokenize.punkt module

tokenized = sent_tokenize(txt)
for i in tokenized:
	
	# Word tokenizers is used to find the words
	# and punctuation in a string
	wordsList = nltk.word_tokenize(i)

	# removing stop words from wordList
	# wordsList = [w for w in wordsList if not w in stop_words]

	# Using a Tagger. Which is part-of-speech
	# tagger or POS-tagger.
	tagged = nltk.pos_tag(wordsList)

	print(tagged)


[('The', 'DT'), ('old', 'JJ'), ('ash', 'NN'), ('tree', 'NN'), ('near', 'IN'), ('the', 'DT'), ('river', 'NN'), ('stood', 'VBD'), ('tall', 'JJ'), ('and', 'CC'), ('slender', 'NN')]


In [ ]:
from nltk.corpus import wordnet as wn
from numpy import dot
from numpy.linalg import norm
import numpy as np

sample_sent = "Water overflowed bank"
test_data = sample_sent.split(" ")
# test_data = ["school", "seeing", "return", "students"]
ce = get_context_embedding(test_data)
sentence_tags = []

for i in range(len(test_data)):
  senses = get_senses(test_data[i])
  sense_emb_list = get_sense_embedding(senses)

  best_sense = ''
  max_cos_sim = 0
  for sense, sense_embedding in sense_emb_list.items():  
    cos_sim = dot(sense_embedding,ce)/(norm(sense_embedding)*norm(ce))
    if cos_sim > max_cos_sim:
      best_sense = sense
      max_cos_sim = cos_sim

  sentence_tags.append(best_sense)

for i in range(len(sentence_tags)):
  print(test_data[i], sentence_tags[i], "\t-\t", wn.synset(sentence_tags[i]).definition())

Water body_of_water.n.01 	-	 the part of the earth's surface covered with water (such as a river or lake or ocean)
overflowed bubble_over.v.01 	-	 overflow with a certain feeling
bank deposit.v.02 	-	 put into a bank account


In [ ]:
from nltk.corpus import wordnet as wn
from numpy import dot
from numpy.linalg import norm
import numpy as np

ce1 = get_context_embedding(["went", "bank", "withdraw", "some", "money"])
ce2 = get_context_embedding(["went", "bank", "have", "bath", "river"])
ce3 = get_context_embedding(["school", "seeing", "return", "students"])
ce4 = get_context_embedding(["school", "fish", "swimming", "past", "island"])

sentence = ""
ce5 = get_context_embedding([])

# s = get_senses("bank", 'n')
s = get_senses("school", 'n')
# s = ['bank.n.01', 'depository_financial_institution.n.01', 'bank.n.03', 'bank.n.04', 'bank.n.05', 'bank.n.06', 'bank.n.07', 'savings_bank.n.02', 'bank.n.09', 'bank.n.10']
# s = ['school.n.01', 'school.n.02', 'school.n.03', 'school.n.04', 'school.n.05', 'school.n.06', 'school.n.07']
se = get_sense_embedding(s)

best_sense = ''
max_cos_sim = 0
for sense, sense_embedding in se.items():  
    cos_sim = dot(sense_embedding, ce4)/(norm(sense_embedding)*norm(ce4))
    if cos_sim > max_cos_sim:
      best_sense = sense
      max_cos_sim = cos_sim

print(s)
print(best_sense)
print(wn.synset(best_sense).definition())
print(wn.synset(best_sense).examples())

In [ ]:
true_tag_sequences = [[tag for (_, tag) in sentence] for sentence in filtered_sentences]

true_tag_sequences[59]

actual_tags = []
for sequence in true_tag_sequences:
  tag_seq = []
  for tag in sequence:
    if tag.startswith("Lemma'"):
      x = tag[6:].split(".")
      tag_seq.append('.'.join(x[:-1]))
    else:
      tag_seq.append(tag)
  actual_tags.append(tag_seq)
#   test_sentences.append(test_sentence)

len(actual_tags)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(actual_tags, predicted_tags)
print(report)

In [ ]:
def calculate_precision_recall_F1(predicted_tags, actual_tags):
  true_positives = 0
  for i in range(len(actual_tags)):                 # i - sentence
    for j in range(len(actual_tags[i])):            # j - tag
      if(predicted_tags[i][j] == actual_tags[i][j]) and (predicted_tags[i][j] != ''):
        true_positives = true_positives + 1

  total_predicted_tags = 0
  for sentence in predicted_tags:
    for tag in sentence:
      if(tag != ''):
        total_predicted_tags += 1

  total_actual_tags = 0
  for sentence in actual_tags:
    for tag in sentence:
      if(tag != ''):
        total_actual_tags += 1

  precision = true_positives/total_predicted_tags
  recall = true_positives/total_actual_tags
  F1_score = 2*precision*recall/(precision+recall)

  return precision, recall, F1_score

calculate_precision_recall_F1(predicted_tags, actual_tags)

In [ ]:
tags = set()
count = 0
for sentence in filtered_sentences:
  for word, tag in sentence:
    if word.startswith('NE'):
      # print(word, tag)
      tags.add(tag)
      
    if tag.startswith('NE'):
      # print(word)
      count += 1

print(count)

In [ ]:
tags = set()
for sentence in filtered_sentences:
  for word, tag in sentence:
    words = word.split(" ")
    if len(words) > 1 and words[0] != 'NE':
      print(words, tag)
tags

In [ ]:
pos_tags = set()
for sentence in filtered_sentences:
  for word, tag in sentence:
    tag.split(" ")

In [ ]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn
wn.synset('in_the_first_place.r.02').examples()

wn.synsets('black')
print(wn.synset("black.a.02").definition())
print(wn.synset("black.a.02").examples())

# model_w2v['NE']